In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU Name: NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
# hyperparameters
INPUT_SIZE = 1
SEQUENCE_LENGTH = 60
HIDDEN_SIZE = 64
NUM_LAYERS = 2
NUM_CLASSES = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 16
NUM_EPOCHS = 100

In [3]:
# dataset class

class SonarDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file, header=None)
        
        X = df.iloc[:, 0:60].values.astype(np.float32)
        y = df.iloc[:, 60].values

        self.label_encoder = LabelEncoder()
        y = self.label_encoder.fit_transform(y) 
        
        self.X = torch.tensor(X).unsqueeze(-1) 
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
# loading the dataset

try:
    full_dataset = SonarDataset('sonar.all-data.csv')
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'sonar.all-data.csv' not found. Please ensure the file is in the directory.")

Data loaded successfully.


In [6]:
# implementing RDNN

class RDNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RDNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.rnn(x, h0)
        
        out = out[:, -1, :]
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

model = RDNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(device)
print(model)

RDNN(
  (rnn): RNN(1, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [7]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [8]:
# training loop

print("Starting Training...")
model.train()
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}')

Starting Training...
Epoch [10/100], Loss: 0.7140
Epoch [20/100], Loss: 0.6808
Epoch [30/100], Loss: 0.5864
Epoch [40/100], Loss: 0.7759
Epoch [50/100], Loss: 0.6603
Epoch [60/100], Loss: 0.6469
Epoch [70/100], Loss: 0.7280
Epoch [80/100], Loss: 0.6816
Epoch [90/100], Loss: 0.7677
Epoch [100/100], Loss: 0.6344


In [9]:
# evaluation

print("Starting Evaluation...")
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

Starting Evaluation...
Accuracy:  0.3492
Precision: 0.3492
Recall:    1.0000
F1 Score:  0.5176
